In [7]:
import pandas as pd
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences
import lightgbm as lgb
from sklearn.base import BaseEstimator
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor


In [8]:
df = pd.read_csv('../data/SBER_250914_250915.csv', sep=';')

df["date_str"]=df["<DATE>"].apply(lambda x: str(x).zfill(6)).astype(str)
df["time_str"]=df["<TIME>"].apply(lambda x: str(x).zfill(6)).astype(str)
df["datetime_str"]=df["date_str"]+df["time_str"]
df["ds"]= pd.to_datetime(df["datetime_str"], format='%y%m%d%H%M%S', utc=True)
df["unique_id"] = "SBER"

df.drop(["date_str", "time_str", "datetime_str", "<DATE>", "<TIME>", "<OPEN>", "<HIGH>", "<LOW>", "<VOL>"], axis=1, inplace=True)
df.rename(columns={"<CLOSE>": "y"}, inplace=True)

df = df.tail(200)

In [9]:
class Naive(BaseEstimator):
    def fit(self, X, y):
        return self

    def predict(self, X):
        return X['lag1']

In [10]:
# Model dictionary
models ={
        'naive': Naive(),
        'lgbm': lgb.LGBMRegressor(verbosity=-1),
        'lasso': Lasso(),
        'lin_reg': LinearRegression(),
        'ridge': Ridge(),
        'knn': KNeighborsRegressor(),
        'mlp': MLPRegressor(),
        'rf': RandomForestRegressor()
    }

In [11]:
fcst = MLForecast(
    models=models,
    freq='min',
    lags=[1],
    target_transforms=[Differences([1])],
)

In [12]:
#fcst.preprocess(df).head(2)

In [13]:
fcst.fit(df)

MLForecast(models=[naive, lgbm, lasso, lin_reg, ridge, knn, mlp, rf], freq=min, lag_features=['lag1'], date_features=[], num_threads=1)

In [14]:
preds = fcst.predict(1)
preds

,unique_id,ds,naive,lgbm,lasso,lin_reg,ridge,knn,mlp,rf
0,SBER,2025-09-15 15:08:00+00:00,302.66,302.575349,302.551465,302.573419,302.5642,302.55,302.545814,302.584988
